In [40]:
library("RPostgreSQL")
library('rgeos')
library('sp')

drv <- dbDriver("PostgreSQL")
con <- dbConnect(drv, dbname="apps", host="db.dev.mapc.org", port="5432", user="editor", password="M999PCedit.451")

#lookup table
lookup = dbGetQuery(con,"select b25003001, vacancy_ra, renter_sha, listings from listings_weighted WHERE listings > 0")

# fit <- lm(y ~ x1 + x2 + x3, data=filtered)
# summary(fit) # show results

get_bigrams <- function(str)
{
  lstr = tolower(str)
  bigramlst = list()
  for(i in 1:(nchar(str)-1))
  {
    bigramlst[[i]] = substr(str, i, i+1)
  }
  return(bigramlst)
}

str_similarity <- function(str1, str2)
{
   pairs1 = get_bigrams(str1)
   pairs2 = get_bigrams(str2)
   unionlen  = length(pairs1) + length(pairs2)
   hit_count = 0
   for(x in 1:length(pairs1)){
        for(y in 1:length(pairs2)){
            if (pairs1[[x]] == pairs2[[y]])
                hit_count = hit_count + 1
        }
   }
   return ((2.0 * hit_count) / unionlen)
}

In [39]:
filtered <- lookup
filtered <- filtered[complete.cases(filtered),]
filtered <- lookup[lookup$b25003001 > 0 & lookup$listings != "NA",]
filtered$listings_norm = ((filtered$listings-min(filtered$listings))) / (max(filtered$listings)/ min(filtered$listings))
fit <- lm(listings ~ vacancy_ra + renter_sha, data=filtered)
summary(fit) # show results


Call:
lm(formula = listings ~ vacancy_ra + renter_sha, data = filtered)

Residuals:
    Min      1Q  Median      3Q     Max 
-107.94  -35.44  -10.67    7.69  476.64 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -16.389      5.329  -3.075  0.00218 ** 
vacancy_ra    83.074    124.336   0.668  0.50425    
renter_sha   118.629     11.810  10.045  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 70.14 on 745 degrees of freedom
Multiple R-squared:  0.1526,	Adjusted R-squared:  0.1503 
F-statistic: 67.07 on 2 and 745 DF,  p-value: < 2.2e-16


In [20]:
listings = dbGetQuery(con,"select ask,bedrooms,title from public.listings")

In [24]:
listings$bedroom_norm = listings$bedrooms
listings$bedroom_norm[listings$bedroom_norm==0] <- 1
listings$per_bedroom = listings$ask / listings$bedroom_norm

[1] 18403

In [23]:
# plot(listings$bedrooms, listings$per_bedroom)
# summary(listings[listings$bedrooms == 1,]$per_bedroom, na.rm = TRUE)
# sd(listings[listings$bedrooms == 1,]$per_bedroom, na.rm = TRUE)

In [60]:
# listings[with(listings, order(title)),]
nrow(listings[!duplicated(listings$title),])

[1] 27032